In [1]:
import pandas as pd
import numpy as np
import re
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset,DataLoader

from numpy import array
from numpy import asarray
from numpy import zeros
from keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.layers import Flatten
from keras.layers import Embedding

import random
import os

from sklearn.metrics import f1_score,accuracy_score, confusion_matrix

from sklearn.model_selection import train_test_split

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
device

/home/hyj/anaconda3/envs/bert/lib/python3.7/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


device(type='cuda')

In [2]:
params = {
    'EPOCHS':5,
    'LEARNING_RATE':2e-5,
    'BATCH_SIZE':256,
    'SEED':45,
    'DATA_PATH': '../../make_data/preprocessed_data/good_bad_df.csv',
    'SAVE_PATH':'../Models/Glove_sentence.pt',
    'max_length':512
}

In [3]:
def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(params['SEED']) # Seed 고정

In [4]:
data = pd.read_csv(params['DATA_PATH'],index_col=[0])

data.loc[data['ad_label']==1,'ad_label']=0
data.loc[data['ad_label']==2,'ad_label']=1

train_data, test_data = train_test_split(data,test_size=0.2,random_state=params['SEED'],shuffle=True)
valid_data, test_data = train_test_split(test_data,test_size=0.5,random_state=params['SEED'],shuffle=True)

train_data = train_data.sample(frac=1).reset_index(drop=True)
valid_data = valid_data.sample(frac=1).reset_index(drop=True)
test_data = test_data.sample(frac=1).reset_index(drop=True)

train_data.reset_index(drop=True,inplace=True)
valid_data.reset_index(drop=True,inplace=True)
test_data.reset_index(drop=True,inplace=True)

In [5]:
print(len(train_data))
print(len(train_data.drop_duplicates(['summary'])))

print(len(valid_data))
print(len(valid_data.drop_duplicates(['summary'])))

print(len(test_data))
print(len(test_data.drop_duplicates(['summary'])))

print(train_data['ad_label'].value_counts()/len(train_data))
print(valid_data['ad_label'].value_counts()/len(valid_data))
print(test_data['ad_label'].value_counts()/len(test_data))

6867
6867
858
858
859
859
0    0.696665
1    0.303335
Name: ad_label, dtype: float64
0    0.715618
1    0.284382
Name: ad_label, dtype: float64
0    0.704307
1    0.295693
Name: ad_label, dtype: float64


In [6]:
t = Tokenizer()
t.fit_on_texts(train_data['summary'])
vocab_size = len(t.word_index) + 1

In [7]:
train_docs = t.texts_to_sequences(train_data['summary'])
valid_docs = t.texts_to_sequences(valid_data['summary'])
test_docs = t.texts_to_sequences(test_data['summary'])

In [8]:
# pad documents to a max length of 4 words
max_length = 530
train_docs = pad_sequences(train_docs, maxlen=max_length, padding='post')
valid_docs = pad_sequences(valid_docs, maxlen=max_length, padding='post')
test_docs = pad_sequences(test_docs, maxlen=max_length, padding='post')

In [9]:
# load the whole embedding into memory
embeddings_index = dict()
f = open('glove.txt')
for line in f:
	values = line.split()
	word = values[0]
	coefs = asarray(values[1:], dtype='float32')
	embeddings_index[word] = coefs
f.close()
print('Loaded %s word vectors.' % len(embeddings_index))

Loaded 358043 word vectors.


In [10]:
embedding_matrix = zeros((vocab_size, 100))
for word, i in t.word_index.items():
	embedding_vector = embeddings_index.get(word)
	if embedding_vector is not None:
		embedding_matrix[i] = embedding_vector

In [11]:
import tensorflow as tf
from sklearn.metrics import f1_score

import tensorflow as tf
from sklearn.metrics import f1_score

class F1Score(tf.keras.metrics.Metric):
    def __init__(self, name='f1_score', **kwargs):
        super(F1Score, self).__init__(name=name, **kwargs)
        self.true_positives = self.add_weight(name='tp', initializer='zeros')
        self.false_positives = self.add_weight(name='fp', initializer='zeros')
        self.false_negatives = self.add_weight(name='fn', initializer='zeros')

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.cast(y_true, tf.float32)
        y_pred = tf.cast(tf.round(y_pred), tf.float32)

        tp = tf.reduce_sum(y_true * y_pred)
        fp = tf.reduce_sum((1 - y_true) * y_pred)
        fn = tf.reduce_sum(y_true * (1 - y_pred))

        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

    def result(self):
        precision = self.true_positives / (self.true_positives + self.false_positives + tf.keras.backend.epsilon())
        recall = self.true_positives / (self.true_positives + self.false_negatives + tf.keras.backend.epsilon())
        f1 = 2 * precision * recall / (precision + recall + tf.keras.backend.epsilon())
        return f1

    def reset_states(self):
        # 변수를 초기화합니다.
        self.true_positives.assign(0.0)
        self.false_positives.assign(0.0)
        self.false_negatives.assign(0.0)

In [12]:
# define model
model = Sequential()
e = Embedding(vocab_size, 100, weights=[embedding_matrix], input_length=530, trainable=False)
model.add(e)
model.add(Flatten())
model.add(Dense(15000, input_dim=53000))
model.add(Dropout(0.5))
model.add(Dense(128, input_dim=15000))
model.add(Dropout(0.3))
model.add(Dense(1, activation='sigmoid'))
# compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy',F1Score()])
# summarize the model
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 530, 100)          2157500   
                                                                 
 flatten (Flatten)           (None, 53000)             0         
                                                                 
 dense (Dense)               (None, 15000)             795015000 
                                                                 
 dropout (Dropout)           (None, 15000)             0         
                                                                 
 dense_1 (Dense)             (None, 128)               1920128   
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 


2023-06-21 10:42:36.883483: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-21 10:42:36.884384: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-21 10:42:36.884504: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2023-06-21 10:42:36.884737: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags

 dense_2 (Dense)             (None, 1)                 129       
                                                                 
Total params: 799,092,757
Trainable params: 796,935,257
Non-trainable params: 2,157,500
_________________________________________________________________
None


In [13]:
# fit the model
model.fit(train_docs, train_data['ad_label'], epochs=params['EPOCHS'], verbose=2)


Epoch 1/5


2023-06-21 10:42:39.091137: I tensorflow/stream_executor/cuda/cuda_blas.cc:1786] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


215/215 - 10s - loss: 3.4558 - accuracy: 0.7951 - f1_score: 0.6556 - 10s/epoch - 44ms/step
Epoch 2/5


/home/hyj/anaconda3/envs/bert/lib/python3.7/site-packages/keras/engine/training.py:2086: UserWarning: Metric F1Score implements a `reset_states()` method; rename it to `reset_state()` (without the final "s"). The name `reset_states()` has been deprecated to improve API consistency.
  m.reset_state()


215/215 - 9s - loss: 0.9637 - accuracy: 0.8965 - f1_score: 0.8265 - 9s/epoch - 40ms/step
Epoch 3/5
215/215 - 9s - loss: 0.1950 - accuracy: 0.9422 - f1_score: 0.9042 - 9s/epoch - 40ms/step
Epoch 4/5
215/215 - 9s - loss: 0.0951 - accuracy: 0.9640 - f1_score: 0.9403 - 9s/epoch - 40ms/step
Epoch 5/5
215/215 - 9s - loss: 0.0758 - accuracy: 0.9729 - f1_score: 0.9551 - 9s/epoch - 40ms/step


In [14]:
loss, accuracy, f1 = model.evaluate(train_docs, train_data['ad_label'])
print('Accuracy: %f' % (accuracy),'F1_Score: %f' % (f1))

215/215 [==============================] - 1s 6ms/step - loss: 0.0492 - accuracy: 0.9818 - f1_score: 0.9696
Accuracy: 0.981797 F1_Score: 0.969623


In [15]:
loss, accuracy, f1 = model.evaluate(valid_docs, valid_data['ad_label'], verbose=1)
print('Accuracy: %f' % (accuracy),'F1_Score: %f' % (f1))

 1/27 [>.............................] - ETA: 0s - loss: 0.0328 - accuracy: 1.0000 - f1_score: 1.0000

27/27 [==============================] - 0s 6ms/step - loss: 0.3853 - accuracy: 0.9033 - f1_score: 0.8282
Accuracy: 0.903263 F1_Score: 0.828157


In [16]:
loss, accuracy, f1 = model.evaluate(test_docs, test_data['ad_label'], verbose=0)
print('Accuracy: %f' % (accuracy),'F1_Score: %f' % (f1))

Accuracy: 0.916182 F1_Score: 0.856000
